# Code Interpreter Toolkit with LangGraph React Agent

This notebook demonstrates how to use the AWS Code Interpreter Toolkit with a LangGraph React Agent to perform code execution tasks. The Code Interpreter Toolkit provides a set of tools for running code, executing shell commands, and managing files in a secure environment.

## Setup and Installation

First, make sure you have the required packages installed:

In [ ]:
%pip install -q langgraph langchain-aws

## Import Required Libraries

In [1]:
import asyncio
import logging
from typing import Dict, List, Any

from langchain.chat_models import init_chat_model
from langchain_aws.tools import create_code_interpreter_toolkit
from langgraph.prebuilt import create_react_agent

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Create Code Interpreter Toolkit

The Code Interpreter Toolkit provides tools for running code, managing files, and executing shell commands in a secure environment managed by AWS.

In [2]:
# Create the code interpreter toolkit
# This is an async function as it sets up the tools
toolkit, code_tools = await create_code_interpreter_toolkit(region="us-west-2")

# Display available tools
print(f"Available code interpreter tools: {[tool.name for tool in code_tools]}")

Available code interpreter tools: ['execute_code', 'execute_command', 'read_files', 'list_files', 'delete_files', 'write_files', 'start_command_execution', 'get_task', 'stop_task']


## Initialize LLM

Set up the language model that will power our agent. We'll use Claude 3.5 Haiku through Bedrock.

In [3]:
# Initialize the language model using bedrock_converse provider
provider = "bedrock_converse"
model_id = "us.anthropic.claude-3-5-haiku-20241022-v1:0"
model_with_provider = f"{provider}:{model_id}"

# Create the model instance
model = init_chat_model(model_with_provider)

## Create React Agent

Now we'll create a React agent using LangGraph's prebuilt agent. The React agent uses a reasoning and acting approach to solve tasks step by step.

In [4]:
# Create the React agent with code interpreter tools
agent = create_react_agent(
    model=model,
    tools=code_tools,
    # Customize the agent prompt for code execution tasks
    prompt="""You are a code execution assistant that can run Python code, execute shell commands, and manage files.
    Use the available code interpreter tools to complete programming tasks.
    
    Available tools:
    - execute_code: Run code in the environment (primarily Python)
    - execute_command: Run shell commands
    - read_files: Read content of files
    - list_files: List files in directories
    - delete_files: Remove files
    - write_files: Create or update files
    - start_command_execution: Start long-running commands asynchronously
    - get_task: Check status of async tasks
    - stop_task: Stop running tasks
    
    Follow these steps for each task:
    1. Understand the problem requirements
    2. Write code or commands to solve the problem
    3. Execute the code and analyze results
    4. Refine your approach based on results if needed
    5. Provide a clear explanation of the solution"""
)


## Execute the Agent

Now let's run the agent on a code execution task. We'll set up a function to execute tasks and print the results.

In [5]:
async def run_code_interpreter_agent(query: str, session_id: str = "code_session1"):
    """
    Run the code interpreter agent on a specific query with session tracking
    
    Args:
        query: The task to perform
        session_id: Unique identifier for the code interpreter session
        
    Returns:
        Agent response
    """
    try:
        # Configure the session ID for thread-aware tools
        config = {
            "configurable": {
                "thread_id": session_id
            }
        }
        
        # Invoke the agent with the query
        result = await agent.ainvoke(
            {
                "messages": [{
                    "role": "human",
                    "content": query
                }]
            },
            config=config
        )
        
        return result
    except Exception as e:
        logger.error(f"Error running code interpreter agent: {e}")
        raise

## Example Tasks

Let's run some example code execution tasks to demonstrate the agent's capabilities.

In [6]:
# Example 1: Execute basic Python code
task1 = "Create a Python function that calculates the factorial of a number, then test it with n=5"

result1 = await run_code_interpreter_agent(task1)
print("Task 1 Result:")
print(result1["messages"][-1].content)

INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.tools.code_interpreter_toolkit:Started code interpreter with session_id:01K07JJWKRSDPHDDC8J03A0VSJ for thread:code_session1
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Task 1 Result:
Let me explain the code and results:

1. I created a `factorial()` function that:
   - Handles negative number input with a `ValueError`
   - Has base cases for 0 and 1 (both return 1)
   - Uses recursive calculation for numbers > 1

2. Testing with n=5:
   - 5! (5 factorial) = 5 × 4 × 3 × 2 × 1 = 120 ✓
   - The function correctly calculated 120

3. Additional test cases:
   - Factorial of 0 is 1
   - Factorial of 1 is 1
   - Factorial of 3 is 6 (3 × 2 × 1)

The function works as expected. I've also included error handling for negative numbers (commented out to prevent stopping execution).

Would you like me to demonstrate the error handling or explain the recursive approach in more detail?

Note: For very large numbers, this recursive approach might cause a stack overflow. An iterative approach or using `math.factorial()` would be more efficient for large inputs.


In [7]:
# Example 2: Create and manipulate files
task2 = """Create a CSV file named 'data.csv' with the following data:
Name,Age,City
John,30,New York
Alice,25,Boston
Bob,35,Chicago
Emma,28,Seattle

Then write a Python script to read this CSV file and calculate the average age."""

result2 = await run_code_interpreter_agent(task2)
print("Task 2 Result:")
print(result2["messages"][-1].content)

INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Task 2 Result:
The task is complete! I've:
1. Created a CSV file named 'data.csv'
2. Written a Python script to read the file
3. Calculated the average age (29.50)
4. Verified the file contents

Is there anything else you would like me to do with this data?


In [9]:
# Example 3: Generate plots
task3 = """Generate a matplotlib plot showing a sine wave over the range [0, 2π] and save it as 'sine_wave.png'. 
Then create another plot showing both sine and cosine waves on the same graph and save it as 'trig_functions.png'."""

result3 = await run_code_interpreter_agent(task3)
print("Task 3 Result:")
print(result3["messages"][-1].content)

INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Task 3 Result:
Great! The plots have been successfully generated and saved. The `sine_wave.png` shows the sine function from 0 to 2π, and `trig_functions.png` displays both sine and cosine waves on the same graph, making it easy to compare their behaviors.

Is there anything else you would like me to do with these plots or any other visualizations you're interested in?


In [10]:
task4 = "Show me all the files present in the code interpreter environment"

result4 = await run_code_interpreter_agent(task4)
print("Task 4 Result:")
print(result4["messages"][-1].content)

INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Task 4 Result:
Great! Let me break down the files in the current directory:

1. Directories:
   - `.cache`
   - `.config`
   - `.ipython`
   - `log`

2. Files:
   - `data.csv`
   - `sine_wave.png`
   - `trig_functions.png`

There are a few interesting things to note:
- There's a CSV file called `data.csv`
- Two image files: `sine_wave.png` and `trig_functions.png`
- Some hidden configuration directories

Would you like me to show you the contents of any of these files or provide more details about them?


## Clean Up Resources

Always clean up code interpreter resources when done to avoid unnecessary resource usage.

In [11]:
# Clean up code interpreter resources
await toolkit.cleanup()
print("Code interpreter resources cleaned up successfully")

INFO:langchain_aws.tools.code_interpreter_toolkit:All code interpreter sessions cleaned up


Code interpreter resources cleaned up successfully
